In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from keras import backend as K

# Keras imports
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.metrics import categorical_accuracy, binary_accuracy
from keras.optimizers import Adam, SGD, Nadam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate

# imports from files
from src.metrics import dice_coef, jacc_coef, jacc_loss, dice_loss
from src.preprocessing import build_matrix_preprocess, build_matrix_mask
from src.models import get_unet, u_net, u_net_normalized

Using TensorFlow backend.


# Preprocessing

In [0]:
# initialization
nb_training_images = 700
nb_validation_images = 200
nb_test_images = 100

# Resize image dimension
img_rows, img_cols = 128, 128

In [0]:
# train data is stored in train.csv
df = pd.read_csv('./data/train.csv')
X_df = df['ImageId']
X = X_df.values

# test and validation data are stored in test.svc
df_submission = pd.read_csv('./data/test.csv')
test = df_submission.values

In [0]:
# process and import train images and corresponding masks
imgs_train = build_matrix_preprocess(X)
imgs_mask_train = build_matrix_mask(X)

# process and import valdiation and test images and corresponding masks
imgs_test = build_matrix_preprocess(test)
imgs_mask_test = build_matrix_mask(test)
# NB : The above functions take time to compute the matrices

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [0]:
# reshape size to add extra dimension
imgs_mask_train = imgs_mask_train.reshape(nb_training_images, height, width, 1)
imgs_mask_test = imgs_mask_test.reshape(nb_validation_images + nb_test_images, height, width,1)

In [0]:
# Assign train, validation and test image and mask based on training dataset 
x_train = imgs_train
y_train = imgs_mask_train

x_val = imgs_test[:nb_validation_images]
y_val = imgs_mask_test[:nb_validation_images]

x_test = imgs_test[nb_validation_images:]
y_test = imgs_mask_test[nb_validation_images:]

# Settings

In [0]:
# Model parameters
model_name = "model1"
seed = 1
nb_epoch = 30
batch_size = 32
loss_param = 'dice'
optimizer_param = 'adam'
monitor_metric = 'val_jacc_coef'
metrics = [jacc_coef]

# Loss options    
loss_options = {'BCE': 'binary_crossentropy', 'dice':dice_loss, 'jacc':jacc_loss}
loss = loss_options[loss_param]

# Optimizer options
optimizer_options = {'adam': Adam(lr=1e-5),
                     'sgd': SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)}
optimizer = optimizer_options[optimizer_param]

# Specify model filename
model_filename = "u_net.h5"

# Callbacks

In [0]:
from keras.callbacks import ModelCheckpoint
checkpoint_path="./u_net.hdf5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor= monitor_metric, verbose=1, save_best_only=True, mode='max')
save_best_only = [checkpoint]

#Data generator

In [0]:
def myGenerator(train_generator, train_mask_generator):
    while True:
        train_gen = next(train_generator)
        train_mask = next(train_mask_generator)
        yield (train_gen, train_mask)

In [0]:
# Define dictionary for data augmentation
data_gen_args = dict(featurewise_center = False, 
                         featurewise_std_normalization = False, 
                         samplewise_std_normalization = False, 
                         zca_whitening = False, 
                         rotation_range = 270, 
                         width_shift_range = 0.1, 
                         height_shift_range = 0.1, 
                         horizontal_flip = False, 
                         vertical_flip = False, 
                         zoom_range = 0.2,
                         channel_shift_range = 0,
                         fill_mode = 'reflect',
                         dim_ordering = K.image_dim_ordering())
data_gen_mask_args = dict(list(data_gen_args.items()) + list({'fill_mode':'nearest','samplewise_center':False}.items()))

In [0]:
# Perform data augmentation using Keras ImageDataGenerator
batch_size = 32
seed=1
print("Create Data Generator")
train_datagen = ImageDataGenerator(data_gen_args)
train_datagen.fit(x_train)
train_mask_datagen = ImageDataGenerator(data_gen_mask_args)
train_mask_datagen.fit(y_train)
train_generator = train_datagen.flow(x_train, batch_size=batch_size, seed=seed)
train_mask_generator = train_mask_datagen.flow(y_train, batch_size=batch_size, seed=seed)
train_generator_f = myGenerator(train_generator, train_mask_generator)

Create Data Generator


#Specify network architecture for training


### U-net with batch normalization

In [0]:
def u_net_normalized(img_rows, img_cols, loss , optimizer, metrics):
    
    # Convolution Layers (BatchNorm after non-linear activation)
  
    img_input = Input((img_rows, img_cols,3))
    x = Conv2D(16, (3, 3), padding='same', name='conv1')(img_input)
    x = BatchNormalization(name='bn1')(x)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv3')(x)
    x = BatchNormalization(name='bn3')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv4')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv5')(x)
    x = BatchNormalization(name='bn5')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (4, 4), padding='same', name='conv6')(x)
    x = BatchNormalization(name='bn6')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = BatchNormalization(name='bn7')(x)
    x = Dropout(0.5)(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv8')(x)
    x = BatchNormalization(name='bn8')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = BatchNormalization(name='bn9')(x)
    x = Activation('relu')(x)
    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)
  
  # Deconvolution Layers (BatchNorm after non-linear activation)
  
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv1')(x)
    x = BatchNormalization(name='bn19')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv2')(x)
    x = BatchNormalization(name='bn12')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv3')(x)
    x = BatchNormalization(name='bn13')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (4, 4), padding='same', name='deconv4')(x)
    x = BatchNormalization(name='bn14')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv5')(x)
    x = BatchNormalization(name='bn15')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv6')(x)
    x = BatchNormalization(name='bn16')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(32, (3, 3), padding='same', name='deconv7')(x)
    x = BatchNormalization(name='bn20')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(16, (3, 3), padding='same', name='deconv8')(x)
    x = BatchNormalization(name='bn17')(x)
    x = Dropout(0.5)(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv9')(x)
    x = BatchNormalization(name='bn18')(x)
    x = Activation('sigmoid')(x)
    
    pred = Reshape((img_rows,img_cols,1))(x)
  
    model = Model(inputs=img_input, outputs=pred)
  
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)  
  
    return model

### u_net

In [0]:
def get_unet(img_rows, img_cols, loss , optimizer, metrics):

    inputs = Input((img_rows, img_cols, channels))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer= Adam(lr = 0.003), loss= [jaccard_distance], metrics=[jaccard_coeff])
  
    
    return model

### U-net2 

In [0]:
def u_net2(img_rows, img_cols, loss , optimizer, metrics):
    
    img_input = Input((img_rows, img_cols,3))
    x = Conv2D(16, (3, 3), padding='same', name='conv1')(img_input)
    x = Activation('relu')(x)
    x = Conv2D(32, (3, 3), padding='same', name='conv2')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv3')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv4')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv5')(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (4, 4), padding='same', name='conv6')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
    x = Dropout(0.5)(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv8')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
    x = Activation('relu')(x)
    x = Dense(1024, activation = 'relu', name='fc1')(x)
    x = Dense(1024, activation = 'relu', name='fc2')(x)
  
  # Deconvolution Layers (BatchNorm after non-linear activation)
  
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv1')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv2')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv3')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(128, (4, 4), padding='same', name='deconv4')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv5')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv6')(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(32, (3, 3), padding='same', name='deconv7')(x)
    x = Activation('relu')(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(16, (3, 3), padding='same', name='deconv8')(x)
    x = Dropout(0.5)(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv9')(x)
    x = Activation('sigmoid')(x)
    pred = Reshape((128,128,1))(x)
    model = Model(inputs=img_input, outputs=pred)
  
    model = Model(inputs=img_input, outputs=pred)
  
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)  
  
    return model



# Train

In [0]:
u_net = get_unet(img_rows, img_cols, loss , optimizer, metrics)
history = u_net.fit_generator(train_generator_f,
                             steps_per_epoch = (nb_training_images // batch_size),
                             nb_epoch= nb_epoch, 
                             validation_data=(x_val,y_val),
                             callbacks = save_best_only,
                             verbose = 1) 

# Test

In [0]:
from sklearn.metrics import jaccard_similarity_score

In [0]:
y_pred = u_net.predict(x_test)

In [0]:
print("validation accuracy = ",u_net.evaluate(x_val, y_val))
print("Jaccard similarity score for test data is ",jaccard_similarity_score(y_true, y_pred))

In [0]:
# Show predicted mask and the ground truth
index = 1
predict_input = x_val[index]
ground_truth = y_val[index].reshape(128,128)predictions =model.predict(predict_input.reshape(1,128,128,3), batch_size=1)
prediction = predictions.reshape(128, 128)
plt.figure()
plt.imshow(prediction,cmap='gray')
plt.title('Predicted')
plt.figure()
plt.imshow(ground_truth,cmap='gray')
plt.title('Ground Turth')

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("modelunet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_unet.h5")
print("Saved model to disk")